In [1]:
%matplotlib qt
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["figure.dpi"] = 150
rcParams["savefig.dpi"] = 150

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import corner
import tensorflow as tf
import matplotlib
import hemcee
from hemcee.sampler import TFModel

/home/daniel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


This demo is the same as the one in `tutorial.ipynb`, but using TensorFlow to define the model instead of computing the gradients by hand.

In [4]:
# Generate a random covariance matrix
np.random.seed(42)
ndim = 2
L = np.random.randn(ndim, ndim)
L[np.diag_indices_from(L)] = np.exp(L[np.diag_indices_from(L)])
L[np.triu_indices_from(L, 1)] = 0.0
cov = np.dot(L, L.T)

params = tf.Variable(np.random.multivariate_normal(np.zeros(ndim), cov), dtype=tf.float64)

log_prob = -0.5 * tf.reduce_sum(params * tf.cholesky_solve(L, params[:, None])[:, 0])

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print("Initial log prob: {0}".format(session.run(log_prob)))
    print("Initial gradient: {0}".format(
        session.run(tf.gradients(log_prob, params))))

Initial log prob: -0.054823958751874714
Initial gradient: [array([0.14671612, 0.04226426])]


Then to wrap the model, I would use this syntax:

In [5]:
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

model = TFModel(log_prob, var_list=[params])
model.setup()

coords = model.current_vector()
print("Model value: {0}".format(model.value(coords)))
print("Model gradient: {0}".format(model.gradient(coords)))

Model value: -0.054823958751874714
Model gradient: [0.14671612 0.04226426]


The value and gradients here should be the same as the ones that you got above.

Then we set up the sampler using this model:

In [6]:
# Choose a dense metric that we will tune
#metric = hemcee.metric.DenseMetric(np.eye(ndim))

# We will also tune the step size
step = hemcee.step_size.VariableStepSize()

# Set up the sampler
sampler = hemcee.NoUTurnSampler(model.value, model.gradient, step_size=step, metric=None)

All Hamiltonian samplers require a tuning phase (often called "warmup" or "burn in").
During this phase, the step size and metric are automatically tuned:

In [7]:
coords = np.random.randn(ndim)
results = sampler.run_warmup(coords, 5000)

initial warm up: step_size: 7.0e+00; mean(accept_stat): 0.503: 100%|██████████| 100/100 [00:01<00:00, 95.60it/s]
warm up 1/8: step_size: 1.5e+00; mean(accept_stat): 0.503: 100%|██████████| 25/25 [00:00<00:00, 113.92it/s]
warm up 2/8: step_size: 6.1e+00; mean(accept_stat): 0.498: 100%|██████████| 25/25 [00:00<00:00, 97.31it/s]
warm up 3/8: step_size: 1.4e+00; mean(accept_stat): 0.508: 100%|██████████| 50/50 [00:00<00:00, 106.91it/s]
warm up 4/8: step_size: 3.1e+00; mean(accept_stat): 0.500: 100%|██████████| 100/100 [00:00<00:00, 100.47it/s]
warm up 5/8: step_size: 2.6e+00; mean(accept_stat): 0.503: 100%|██████████| 200/200 [00:01<00:00, 110.25it/s]
warm up 6/8: step_size: 2.4e+00; mean(accept_stat): 0.502: 100%|██████████| 400/400 [00:03<00:00, 110.58it/s]
warm up 7/8: step_size: 4.2e+00; mean(accept_stat): 0.501: 100%|██████████| 800/800 [00:05<00:00, 149.80it/s]
warm up 8/8: step_size: 3.1e+00; mean(accept_stat): 0.501: 100%|██████████| 3200/3200 [00:17<00:00, 184.85it/s]
final warm u

After burning in, we can run the production MCMC chain:

In [12]:
coords_chain, logprob_chain = sampler.run_mcmc(results[0], 1000, initial_log_prob=results[1], var_names=[None]*5, plot=True)
#coords_chain, logprob_chain = sampler.run_mcmc(coords, 5000, initial_log_prob=None)

step_size: 2.9e+00; mean(accept_stat): 0.601: 100%|██████████| 1000/1000 [00:19<00:00, 51.50it/s]


In [10]:
plt.plot(np.exp(logprob_chain))

In [11]:
#matplotlib.rcParams.update({'font.size': 7})
figure = corner.corner(coords_chain, 
                       quantiles=[0.16, 0.5, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 12})

In [ ]:
taus = np.array([hemcee.autocorr.integrated_time(coords_chain[:, i])[0] for i in range(ndim)])
print("Mean autocorrelation time: {0}".format(np.mean(taus)))